<a href="https://colab.research.google.com/github/frederik-kilpinen/ASDS2/blob/main/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification

In this notebook we train a convolutional neural network to classify images as either fire or koala. The notebook is structured as follows:

1. First we build a scraper to extract pictures of koalas and bush-fires and some other random noise.
2. We build a convolutional neural network

In [1]:
#Necessary Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
import tweepy
import pickle 
import time
import matplotlib.pyplot as plt
import requests as rq
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
#from tf_explain.core.activations import ExtractActivations
from tensorflow.keras.applications.xception import decode_predictions
import os


#from bing_image_downloader import downloader


## Bing Image Scraper

In [ ]:
queries = ["koala", "bush fire"]

In [ ]:
for query in queries:
    downloader.download(query_string, limit=100,  
                        output_dir='image_dataset', 
                        adult_filter_off=False, 
                        force_replace=False, 
                        timeout=60)

## Get Twitter Images

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
image_tweets = pd.read_csv("/content/drive/MyDrive/Python/data/image_tweets", index_col=0)
#Create list of image urls
image_urls = image_tweets["image_url"].unique()

In [ ]:
def image_collect(urls):
    """
    This functions downloads images given a list of image urls.
    --------------------------------------------
    input: list of urls
    returns: images downloaded at specified path
    --------------------------------------------
    """
    import urllib.request
    idx = 0
    images = []
    path = "/content/drive/MyDrive/Python/data/tweet_images/"
    for url in tqdm(urls):
        response = urllib.request.urlretrieve(url, f"{path}img_{idx}.jpg")
        idx += 1

In [ ]:
#Download all the tweet images
image_collect(image_urls[:10])

100%|██████████| 10/10 [00:01<00:00,  5.14it/s]


In [ ]:
os.listdir("/content/drive/MyDrive/Python/data/tweet_images/")

['img_0.jpg', 'img_1.jpg']

In [6]:
###Preprocess the tweet images
images = []
for img in tqdm(os.listdir("/content/drive/MyDrive/Python/data/tweet_images/")):
    if img.endswith(".jpg"):
        path = f"/content/drive/MyDrive/Python/data/tweet_images/{img}"
        img=tf.keras.preprocessing.image.load_img(path, target_size=(299,299))
        img=tf.keras.preprocessing.image.img_to_array(img)
        images.append(img)

images = np.array(images)

100%|██████████| 11/11 [00:02<00:00,  5.13it/s]


In [16]:
images

array([[[[153., 144., 161.],
         [160., 153., 169.],
         [169., 162., 178.],
         ...,
         [238., 242., 245.],
         [237., 240., 245.],
         [236., 241., 245.]],

        [[143., 134., 151.],
         [154., 147., 163.],
         [160., 153., 169.],
         ...,
         [239., 243., 246.],
         [237., 242., 245.],
         [233., 240., 246.]],

        [[150., 141., 158.],
         [162., 155., 171.],
         [169., 162., 178.],
         ...,
         [242., 247., 251.],
         [237., 244., 250.],
         [232., 241., 250.]],

        ...,

        [[181., 153., 129.],
         [177., 150., 123.],
         [177., 147., 121.],
         ...,
         [118.,  96.,  85.],
         [115.,  93.,  80.],
         [112.,  90.,  79.]],

        [[131., 109.,  88.],
         [129., 104.,  84.],
         [128., 101.,  80.],
         ...,
         [105.,  85.,  74.],
         [105.,  86.,  72.],
         [105.,  81.,  71.]],

        [[ 86.,  68.,  54.],
       

In [13]:
images.shape

(10, 299, 299, 3)

In [8]:
images.shape

(10, 299, 299, 3)

## Image classification using pre-trained Keras Xception

In [ ]:
#load pre trained Xception model
model=tf.keras.applications.xception.Xception(weights='imagenet',include_top=True)
#print(model.summary())

In [ ]:
#loading and preprocessing cat image
IMAGE_PATH='./lion.jpg'
img=tf.keras.preprocessing.image.load_img(IMAGE_PATH,target_size=(299,299))
img=tf.keras.preprocessing.image.img_to_array(img)#view the image
plt.imshow(img/255.)